# Image Undistortion using Charuco Board 
Ideally camera calibration is done using [multical](https://github.com/oliver-batchelor/multical) in order to ensure highest compatibility. 

## Multical

We will used [multical](https://github.com/oliver-batchelor/multical) to retrieve the calibration values for the camera. It is recommended to create a separate ```conda``` environment which we will call ```mcal```. This way the calibration software is contained and isn't affected by other packages.   

To do so we first need to follow a few steps:

1. **Create Calibration Pattern:** <br> To do so you can use the accompanied ```checkerboard_charuco_8x11.yaml``` file and running the following command in a terminal: <br> ```multical boards --boards checkerboard_charuco_8x11.yaml --write ~/Documents/ --paper_size_mm 216x279```  <br> This will create a calibration pattern which is recognized by [multical](https://github.com/oliver-batchelor/multical). It is advised to keep the ```checkerboard_charuco_8x11.yaml``` in a known location as it is used for the calibration process later on. It is also a good idea to double check and measure the aruco tag and square width on the print, to match the parameters in the ```checkerboard_charuco_8x11.yaml``` file. If they don't match you can create a copy of the file and change the values to match those of the print. 
2. **Take Pictures of Calibration Pattern:**<br> Now we need to take pictures with the camera that will be used for measuring. Ideally this is done under the same conditions as when measuring. Therefore, if measurement images are taken underwater, with the camera in a casing, the calibration images should be taken under the same circumstances and with ideantical setup. This ensures that the calibration is optimal!
3. **Calibrate Camera:** <br> Once we have the calibration images, these should be stored in a folder with standardized name. Here we will use a folder structure of ```DDMMYYYY/cam1``` (for example: ```31072024/cam1```). Then we can run the calibration for a single camera using [multical](https://github.com/oliver-batchelor/multical) and the following command: <br>```multical calibrate --image_path /home/fritz/Pictures/olympus_undistortion_test/data/checkerboard_images/31072024/ --boards /home/fritz/Downloads/charuco_png_test_8x11.yaml --camera_pattern cam1``` <br> This command will create a few output files in the image directory, of which we will use ```calibration.json``` moving forward.

## Warp Transform using Aruco Tag
This process is also known as rectification and uses a reference shape to transform the image. It is important that this step is done on undistorted images, to reduce error and improve the process. The general process goes as follows:

1. An Aruco Tag is attached to a flat surface on which the objects (i.e. Fish) are photographed and later measured. This tag should always be in full view, as well as the objects in the images. It is important to note the tag number and ideally the dictionary with which it was generated with. 
2. We can then tranform the image to a be as if it was viewed directly from above. This is done by automatically finding the tag in the image and warp transforming it to make the tag square again.
3. Once transformed, the images can be saved for use in measuring the objects of interest!

## Create Aruco Tags

## Read out saved values

newcameraMatrix, distCoeffs, rvecs, tvecs

In [28]:
import glob
import json
import os
import pickle
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
from cv2 import aruco

In [5]:
# Images
filename_glob_pattern = "/media/fritz/01_PNG_DATA/MeasurementImages/26042025/labeled/*.JPG"

# Output path
output_dir = "/media/fritz/01_PNG_DATA/MeasurementImages/26042025/undistorted/"

In [7]:
# Multical
f = open(
    "/media/fritz/01_PNG_DATA/MeasurementImages/calibration_images/tg-6/calibration.json"
)
data = json.load(f)
cameraMatrix = np.array(data["cameras"]["cam0"]["K"])
distCoeffs = np.array(data["cameras"]["cam0"]["dist"]).reshape(1, 5)

In [8]:
images = glob.glob(filename_glob_pattern)

cv2.namedWindow("Undistorted Image", cv2.WINDOW_NORMAL)

if os.path.isdir(output_dir) == False:
    os.makedirs(output_dir)

for img in images:
    filename = output_dir + Path(img).stem + "_undistorted" + Path(img).suffix
    print(filename)
    img = cv2.imread(img)

    # Undistort the image
    h, w = img.shape[:2]
    new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(
        cameraMatrix, distCoeffs, (w, h), 1, (w, h)
    )
    undistorted_image = cv2.undistort(
        img, cameraMatrix, distCoeffs, None, new_camera_matrix
    )

    # Crop the image (if desired, based on ROI)
    x, y, w, h = roi
    undistorted_image = undistorted_image[y : y + h, x : x + w]

    cv2.imwrite(filename, undistorted_image)
    cv2.imshow("Undistorted Image", undistorted_image)
    cv2.waitKey(1)

cv2.destroyAllWindows()

/media/fritz/01_PNG_DATA/MeasurementImages/26042025/undistorted/RB2_26042025_RankNone_P1010001_00000_undistorted.JPG
/media/fritz/01_PNG_DATA/MeasurementImages/26042025/undistorted/RB2_26042025_RankNone_P1010002_00001_undistorted.JPG
/media/fritz/01_PNG_DATA/MeasurementImages/26042025/undistorted/RB2_26042025_Rank1_P1010003_00002_undistorted.JPG
/media/fritz/01_PNG_DATA/MeasurementImages/26042025/undistorted/RB2_26042025_Rank1_P1010004_00003_undistorted.JPG
/media/fritz/01_PNG_DATA/MeasurementImages/26042025/undistorted/RB2_26042025_Rank1_P1010005_00004_undistorted.JPG
/media/fritz/01_PNG_DATA/MeasurementImages/26042025/undistorted/RB2_26042025_Rank1_P1010006_00005_undistorted.JPG
/media/fritz/01_PNG_DATA/MeasurementImages/26042025/undistorted/RB2_26042025_Rank1_P1010007_00006_undistorted.JPG
/media/fritz/01_PNG_DATA/MeasurementImages/26042025/undistorted/RB2_26042025_Rank2_P1010009_00008_undistorted.JPG
/media/fritz/01_PNG_DATA/MeasurementImages/26042025/undistorted/RB2_26042025_Rank2

## Warp Transform using Aruco Tag

In [24]:
import glob

import cv2
import numpy as np


def rectify_image_using_aruco(
    image_path, cameraMatrix, distCoeffs, aruco_dict_type=cv2.aruco.DICT_4X4_100
, draw_tag=False):
    # Load the image
    img = cv2.imread(image_path)

    # Undistort the image
    h, w = img.shape[:2]
    new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(
        cameraMatrix, distCoeffs, (w, h), 1, (w, h)
    )
    undistorted_image = cv2.undistort(
        img, cameraMatrix, distCoeffs, None, new_camera_matrix
    )

    # Crop the image (if desired, based on ROI)
    x, y, w, h = roi
    img = undistorted_image[y : y + h, x : x + w]

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Load the dictionary that was used to generate the markers
    aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)
    # Initialize the detector parameters using default values
    parameters = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

    # Detect the markers in the image
    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(
        gray, aruco_dict, parameters=parameters
    )

    if ids is not None:
        # Assuming we have at least one marker detected, we will use the first detected marker
        first_corners = corners[0]
        corners = corners[0].reshape((4, 2))

        # Define the destination points for the perspective transform
        (top_left, top_right, bottom_right, bottom_left) = corners

        width = max(
            int(np.linalg.norm(bottom_right - bottom_left)),
            int(np.linalg.norm(top_right - top_left)),
        )
        height = max(
            int(np.linalg.norm(top_right - bottom_right)),
            int(np.linalg.norm(top_left - bottom_left)),
        )

        dst_pts = np.array(
            [
                [img.shape[1] - (width - 1), img.shape[0] / 4],
                [img.shape[1], img.shape[0] / 4],
                [img.shape[1], (img.shape[0] / 4) + (height - 1)],
                [img.shape[1] - (width - 1), (img.shape[0] / 4) + (height - 1)],
            ],
            dtype="float32",
        )

        # Get the perspective transform matrix
        M = cv2.getPerspectiveTransform(corners, dst_pts, cv2.WARP_INVERSE_MAP)

        if draw_tag == True:
            # Draw the markers on the frame
            cv2.aruco.drawDetectedMarkers(img, [first_corners], ids[0])
            
        # Apply the perspective transformation to get the rectified image
        rectified_img = cv2.warpPerspective(
            img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR
        )

        return rectified_img
    else:
        print("No ArUco markers detected.")
        return undistorted_image

In [25]:
# Load the predefined dictionary
aruco_dict = cv2.aruco.DICT_4X4_1000
dictionary = cv2.aruco.getPredefinedDictionary(aruco_dict)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

# Measurement images containing charuco tag
charuco_measurement_images = sorted(
    glob.glob(
        "/media/fritz/01_PNG_DATA/MeasurementImages/27042025/labeled/*.JPG"
    )
)

# Output dir
output_dir = "/media/fritz/01_PNG_DATA/MeasurementImages/27042025/undistored_rectified/"
if os.path.isdir(output_dir) == False:
    os.makedirs(output_dir)

# Multical Calibration Parameters
f = open(
    "/media/fritz/01_PNG_DATA/MeasurementImages/calibration_images/tg-6/calibration.json"
)
data = json.load(f)
cameraMatrix = np.array(data["cameras"]["cam0"]["K"])
distCoeffs = np.array(data["cameras"]["cam0"]["dist"]).reshape(1, 5)

In [26]:
for img in charuco_measurement_images:
    frame = cv2.imread(img)

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect markers in the image
    # corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    corners, ids, rejectedImgPoints = detector.detectMarkers(gray)

    # If markers are detected
    if ids is not None:
        rectified_image = rectify_image_using_aruco(img, cameraMatrix, distCoeffs, draw_tag = True)
    else:
        rectified_image = rectify_image_using_aruco(img, cameraMatrix, distCoeffs)

    ## Save to file
    filename = output_dir + Path(img).stem + "_rectified" + Path(img).suffix
    cv2.imwrite(filename, rectified_image)

    cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)
    cv2.imshow("Frame", rectified_image)
    cv2.waitKey(1)
cv2.destroyAllWindows()

No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
No ArUco markers detected.
N

In [ ]:
cv2.destroyAllWindows()

## Foreground Background Segmentation

This is experimental, but in very good images the objects can directly be segmented from the background using this:

In [ ]:
import cv2
from rembg import remove

input_path = "/home/fritz/Pictures/olympus_undistortion_test/data/rectified_images/P7280145_rectified.JPG"
output_path = "/home/fritz/Pictures/fgbgTest.JPG"

input = cv2.imread(input_path)
output = remove(input)
cv2.imwrite(output_path, output)